# KNN Gris

Robots Autónomos. Mapas Topológicos visuales

Autores:
    Alejandro Benítez López, Elena Benito Frey, Mario González Carbayo, Isidro López Dominguez, Blanca Martínez Donoso y Ángel Pavón Pérez

In [1]:
import cv2
import numpy as np
import glob
import os
import matplotlib.pyplot as plt 
import pickle
import pathlib
import random
import statistics as stats

directory = os.getcwd() + "/"

Cargamos un array con todos los frames del video ya clasificados:

In [2]:
try:
    frames = pickle.load(open(directory + "clasificacion_frames.sav", 'rb'))
except:
    print("Falta el fichero \"clasificacion_frames.sav\".")
    print("Se puede generar del \"classify-video-frames.ipynb\"")

Se calculan los histogramas y se normalizan:

In [6]:
# para hacer los calculos mas eficientes se reduce las imagenes a esta resolucion
output_dimension = (848, 480)

# diccionario donde se guardaran todos los histogramas
hist_dict = {}

for i in range(9):
    # cargamos las imagenes de landmark
    filenames = [img for img in glob.glob(directory + "landmarks_img/"+ str(i) + "/*.jpg")]
    filenames.sort()

    images    = [cv2.imread(img,cv2.IMREAD_GRAYSCALE) for img in filenames]
    img_names = [os.path.basename(img)                for img in filenames]
    
    # para todas las imagenes cargadas de este landmark:
    for j, img in enumerate(images):
        # se redimensiona la imagen
        res_img = cv2.resize(img, output_dimension, interpolation=cv2.INTER_CUBIC)
            
        # se calcula el histograma en escala de grises
        hist = cv2.calcHist([img],[0],None,[256],[0,256])
        
        # se calcula el historgama normalizado
        hist_norm = hist / (output_dimension[0] * output_dimension[1])
        

        # se añade al diccionario. La clave es el nombre de la imagen y el valor es su histograma normalizado 
        hist_dict[img_names[j]] = hist_norm 
        
        
    print(f"Landmark {i}. Number Of Histograms: {str(len(hist_dict))}")


Landmark 0. Number Of Histograms: 73
Landmark 1. Number Of Histograms: 135
Landmark 2. Number Of Histograms: 228
Landmark 3. Number Of Histograms: 310
Landmark 4. Number Of Histograms: 371
Landmark 5. Number Of Histograms: 458
Landmark 6. Number Of Histograms: 536
Landmark 7. Number Of Histograms: 599
Landmark 8. Number Of Histograms: 664


Se define la funcion de KNN:

In [4]:
def knn(img, hist_dict):

    # se calcula el histograma en escala de grises
    hist = cv2.calcHist([img],[0],None,[256],[0,256])
    
    # se calcula el historgama normalizado
    hist = hist / (output_dimension[0] * output_dimension[1])
   
    min_distance = 1000000
    min_key = ''
    
    # busca el histograma mas parecido por la distancia euclidea
    for key, value in hist_dict.items():
        
        distance = np.linalg.norm(value-hist)
        
        if distance < min_distance:
            min_distance = distance
            min_key = key
    
    # devuelve el numero del landmark mas cercano
    return int(min_key[9])

## Prueba estática

In [5]:
# se obtienen todas las imagenes de validacion (20% del total de cada landmark)
filenames = [img for img in glob.glob(directory + "landmarks_img/validation/*.jpg")]
if len(filenames) > 0:
    filenames.sort()

    all_img      = [0] * 9
    aciertos_img = [0] * 9

    for f in filenames:
        # se lee la imagen en escala de grises, su nombre y se calcula su histograma con knn()
        img = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
        name = os.path.basename(f)
        landmark = knn(img, hist_dict)

        # se incrementa el uno el total de imagenes del landmark original
        all_img[int(name[9])] +=1

        # solo si se acierta, se incrementa el uno los aciertos en ese landmark
        if int(name[9]) == int(landmark):
            aciertos_img[int(name[9])] += 1
        else:
            print(f"Fallo en {name}")


    # se saca el porcentaje parcial y total:
    porcentaje_total = []

    for i in range(len(all_img)):
        x = (aciertos_img[i] * 100) / all_img[i]
        porcentaje_total.append(x)

    print("Porcentaje por cada landmark:")
    print("\t" + str(porcentaje_total))
    print("Porcentaje medio:")
    print("\t" + str(stats.mean(porcentaje_total)))
    
else:
    print("No hay imagenes en la carpeta de \"validation\"")
    print("Asegurate de generarlo ejecutando \"generate-validation-set-ipynb\"")

Porcentaje por cada landmark:
	[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]
Porcentaje medio:
	100.0


## Prueba dinámica

In [8]:
# se carga el video
cap = cv2.VideoCapture(directory + 'walk-through.mp4')

frame_count = 0

# estadisticas de aciertos
all_img      = [0] * 9
aciertos_img = [0] * 9

while(True):
    
    ret, frame = cap.read()
    
    if ret:
        
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
        
        all_img[frames[frame_count]] += 1
        
        k = cv2.waitKey(1)
        
        if k & 0xFF == ord('q'):
            break
        
        # el video tiene una resolucion de 1920x1080 y nuestro knn esta entrenado con imagenes de 848x480
        # asi que tenemos que redimensionar la imagen
        ri = cv2.resize(frame, output_dimension)
        
        # se usa el knn() para el frame actual
        landmark = knn(ri, hist_dict)
        
        if landmark == frames[frame_count]:
            aciertos_img[landmark] += 1
        
        # se pinta texto en el frame indicando el landmark predecido por el knn
        cv2.putText(ri,  
                'predict landmark = ' + str(landmark),  
                (25, 40),  
                cv2.FONT_HERSHEY_SIMPLEX ,0.7,  
                (0, 255, 255),  
                2,  
                cv2.LINE_4) 
        
        # se pinta texto en el frame indicando el landmark que hemos determinado nosotros para este frame
        cv2.putText(ri,  
                'real landmark = ' + str(frames[frame_count]),  
                (25, 80),  
                cv2.FONT_HERSHEY_SIMPLEX , 0.7,  
                (0, 255, 255),  
                2,  
                cv2.LINE_4) 
               
        cv2.imshow('Prueba dinamica color',ri)
        frame_count +=1
    else:
        break     

cap.release()
cv2.destroyAllWindows()

# se saca el porcentaje parcial y total:
porcentaje_total = []

for i in range(len(all_img)):
    x = (aciertos_img[i] * 100) / all_img[i]
    porcentaje_total.append(x)

print("Porcentaje por cada landmark:")
print("\t" + str(porcentaje_total))
print("Porcentaje medio:")
print("\t" + str(stats.mean(porcentaje_total)))


Porcentaje por cada landmark:
	[77.20465890183029, 90.0489396411093, 24.4258872651357, 98.79518072289157, 80.8411214953271, 96.82539682539682, 93.10344827586206, 38.72053872053872, 96.84763572679509]
Porcentaje medio:
	77.42364528609852
